In [1]:
from pycaret.classification import *
import pandas as pd
import os
import wandb

os.chdir('/home/antoine/gene_pheno_pred')
os.environ["CUDA_VISIBLE_DEVICES"]="1"


wandb: Currently logged in as: antoine-toffano (mlb). Use `wandb login --relogin` to force relogin


In [2]:
# Load training set
train = pd.read_csv('/home/antoine/gene_pheno_pred/2023-05-17_18-04-22_train.csv' , header=0)
test = pd.read_csv('/home/antoine/gene_pheno_pred/2023-05-17_18-04-22_test.csv' , header=0)
val = pd.read_csv('/home/antoine/gene_pheno_pred/2023-05-17_18-04-22_val.csv' , header=0)
merged_val = pd.concat([test, val], axis=0)

merged_val['link'] = merged_val['ground_truth'].apply(lambda x: 1 if x != 'no_link_known' else 0)
merged_val

,0,1,2,3,4,5,6,7,8,9,...,192,193,194,195,196,197,198,199,ground_truth,link
0,-0.140183,-0.029808,-0.203201,-0.013422,-0.056131,0.207323,0.187371,0.170251,-0.190599,-0.196459,...,-0.208771,-0.124182,0.068771,0.145912,0.080039,-0.044662,-0.136030,-0.202819,no_link_known,0
1,0.007257,0.034391,0.040302,-0.014142,0.043601,0.026042,-0.042320,-0.052182,0.007224,0.059336,...,0.007578,0.013686,-0.019377,0.025978,0.045618,0.002407,0.014198,0.041636,http://www.semanticweb.org/needed-terms#001,1
2,-0.017977,0.032711,0.031887,-0.031475,0.039888,0.021578,-0.047201,-0.042827,0.002145,0.049754,...,0.019817,-0.026000,0.029231,0.027440,0.036481,0.023638,-0.003050,0.008508,http://www.semanticweb.org/needed-terms#001,1
3,-0.014240,0.039529,0.029118,-0.017358,0.044518,0.024350,-0.043136,-0.042794,-0.013481,0.052084,...,0.071208,0.044566,0.037520,-0.061345,0.040713,-0.081230,0.059916,0.025145,no_link_known,0
4,-0.084909,0.114381,-0.204285,0.021392,-0.120028,0.203261,0.172854,0.175367,-0.199627,-0.195778,...,0.019101,0.023105,-0.014103,0.006210,-0.004504,0.002287,0.019769,0.032053,http://semanticscience.org/resource/SIO_000281,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234191,0.053616,-0.010338,0.054068,-0.054630,0.033612,-0.074185,-0.056169,0.043632,0.049982,0.065148,...,0.007249,0.027118,-0.006408,0.018064,0.011075,-0.000410,0.022717,0.032426,no_link_known,0
234192,0.059236,-0.030084,0.070195,-0.059798,-0.069581,-0.075291,-0.063509,-0.033916,-0.029080,0.071954,...,0.051774,-0.054903,-0.031336,-0.060177,0.069443,-0.076120,-0.030139,0.058012,no_link_known,0
234193,-0.018654,-0.019158,0.009537,-0.006379,0.017037,-0.006890,-0.028761,-0.017385,0.020085,0.022891,...,0.020308,0.034114,-0.025278,0.003062,0.020570,-0.007069,0.039509,0.040183,http://www.w3.org/2000/01/rdf-schema#subClassOf,1
234194,-0.045248,0.124431,-0.166019,-0.034602,-0.153778,0.156033,0.153976,0.117537,-0.163683,-0.165206,...,0.035035,0.018661,-0.024460,-0.003234,0.029622,-0.009954,0.008429,0.025477,http://www.semanticweb.org/needed-terms#001,1


In [ ]:
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
numeric_cols = merged_val.drop(['ground_truth', 'link'], axis=1)  # Exclude 'ground_truth' and 'link' columns
pca = PCA(n_components=2)  # Specify the number of components you want to keep
pca_result = pca.fit_transform(numeric_cols)
df_pca = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])

In [ ]:
df_pca['link'] = merged_val['link']


In [ ]:
plt.figure(figsize=(10, 6))
colors = ['red', 'blue', 'green']  # Define colors for different 'link' values
for link_value, color in zip(df_pca['link'].unique(), colors):
    indices = df_pca['link'] == link_value
    plt.scatter(df_pca.loc[indices, 'PC1'], df_pca.loc[indices, 'PC2'], c=color, label=link_value)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.title('PCA with Colored Datapoints')
plt.show()
